In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import time
import csv

In [2]:
# Setup
driver = webdriver.Chrome()
driver.get("https://jiji.co.ke/cars?price_min=500000&price_max=644000&sort=price_desc")
time.sleep(3)

# Scroll to load more cars 
for _ in range(200):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3)

In [3]:
# Collect all full listing URLs including query params
base_url = "https://jiji.co.ke"
listing_elements = driver.find_elements(By.CSS_SELECTOR, 'a.b-list-advert-base.qa-advert-list-item')

# Extract full links
car_links = []
print("ðŸ”— Listing Links Found:\n" + "-"*40)
for el in listing_elements:
    href = el.get_attribute("href")
    if href and href.startswith("/"):
        full_url = base_url + href
    elif href and href.startswith("http"):
        full_url = href
    else:
        continue
    # print(full_url)
    car_links.append(full_url)

print(f"\nâœ… Total Listings Found: {len(car_links)}\n")


ðŸ”— Listing Links Found:
----------------------------------------

âœ… Total Listings Found: 1228



In [4]:
# Function to safely extract text
def safe_get(selector, by=By.CSS_SELECTOR):
    try:
        return driver.find_element(by, selector).text.strip()
    except NoSuchElementException:
        return ''

In [5]:
# Prepare CS
with open("scraped_data/car_listings46.csv", "w", newline='', encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["Make", "Model", "Price", "Location", "Year", "Mileage", "Transmission", "Color", "Previous Ownership", "URL"])

    # Loop through car links
    for link in car_links:
        driver.get(link)
        time.sleep(2)

        make = safe_get("div[itemprop='brand']")
        model = safe_get("div[itemprop='model']")
        price = safe_get("span.qa-advert-price-view-value")
        location = safe_get("div.b-advert-info-statistics--region")
        year = safe_get("div[itemprop='productionDate']")
        mileage = safe_get("span[itemprop='mileageFromOdometer']")
        transmission = safe_get("span[itemprop='vehicleTransmission']")
        color = safe_get("div[itemprop='color']")
        ownership = safe_get("span[itemprop='itemCondition']")

        writer.writerow([make, model, price, location, year, mileage, transmission, color, ownership, link])

driver.quit()


ReadTimeoutError: HTTPConnectionPool(host='localhost', port=51710): Read timed out. (read timeout=120)